# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [2]:
import finnhub as fh
API_KEY_FINHUB = "sandbox_cb9s7hqad3i97kdr3fag"
finnhub_client = fh.Client(api_key=API_KEY_FINHUB)

## Importing Our List of Stocks & API Token

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [ ]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [4]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [5]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:

    #Batch query 100 stocks
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        final_dataframe = final_dataframe.append(
            pd.Series([symbol, 
                       data[symbol]['quote']['latestPrice'],
                       data[symbol]['quote']['peRatio'],
                       'N/A'
                       ], 
                      index = my_columns), 
            ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.29,29.52,N/A
1,AAL,14.63,-4.13,N/A
2,AAP,191.16,21.38,N/A
3,AAPL,156.10,25.18,N/A
4,ABBV,157.22,22.31,N/A
...,...,...,...,...
494,YUM,119.22,22.08,N/A
495,ZBH,103.73,106.68,N/A
496,ZBRA,310.50,20.2,N/A
497,ZION,53.11,5.27,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

In [6]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [7]:
final_dataframe


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,36.76,2.25,N/A
1,EBAY,43.65,2.47,N/A
2,AMZN,114.75,2.74,N/A
3,NUE,112.00,3.97,N/A
4,F,12.03,4.2,N/A
5,PVH,59.53,4.3,N/A
6,SYF,31.84,4.4,N/A
7,KSS,28.76,4.42,N/A
8,COF,112.99,4.44,N/A
9,AIG,51.68,4.51,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input`

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:10000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,36.76,2.25,5
1,EBAY,43.65,2.47,4
2,AMZN,114.75,2.74,1
3,NUE,112.00,3.97,1
4,F,12.03,4.2,16
5,PVH,59.53,4.3,3
6,SYF,31.84,4.4,6
7,KSS,28.76,4.42,6
8,COF,112.99,4.44,1
9,AIG,51.68,4.51,3


## Building a Better (and More Realistic) Value Strategy
Filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [13]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
28,ALXN,186.300,N/A,60.13,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,272.600,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,292.280,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,98.610,N/A,49.15,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.85,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
162,ETFC,51.090,N/A,14.43,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
183,FLIR,59.110,N/A,32.61,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
187,FOX,31.110,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FRT,97.670,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
201,GOOG,2280.060,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [14]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100


#Print the entire DataFrame    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.04,N/A,29.87,0.725451,6.97,0.805611,5.7100,0.829659,20.763939,0.803607,10.331207,0.791583,N/A
1,AAL,14.56,N/A,-4.05,0.032064,-1.05,0.06012,0.2764,0.016032,-21.436752,0.008016,0.975838,0.04008,N/A
2,AAP,190.81,N/A,21.63,0.523046,3.98,0.589178,1.0600,0.166333,12.083096,0.43487,2.544190,0.156313,N/A
3,AAPL,152.79,N/A,25.56,0.637275,36.14,0.96994,6.5000,0.853707,19.825406,0.781563,15.234777,0.91984,N/A
4,ABBV,153.68,N/A,22.97,0.56513,17.37,0.933868,4.9100,0.777555,11.889645,0.426854,8.507717,0.685371,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,119.23,N/A,21.45,0.509018,-4.11,0.056112,5.2000,0.803607,19.742561,0.779559,9.321896,0.741483,N/A
495,ZBH,108.68,N/A,103.73,0.981964,1.88,0.283567,2.9700,0.534068,17.439308,0.707415,5.226473,0.392786,N/A
496,ZBRA,301.35,N/A,20.20,0.490982,5.48,0.739479,2.7900,0.50501,15.580490,0.635271,6.383423,0.476954,N/A
497,ZION,52.22,N/A,5.23,0.074148,1.32,0.169339,2.2600,0.420842,4.217492,0.074148,2.108005,0.130261,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.04,N/A,29.87,0.725451,6.97,0.805611,5.7100,0.829659,20.763939,0.803607,10.331207,0.791583,0.791182
1,AAL,14.56,N/A,-4.05,0.032064,-1.05,0.06012,0.2764,0.016032,-21.436752,0.008016,0.975838,0.04008,0.031263
2,AAP,190.81,N/A,21.63,0.523046,3.98,0.589178,1.0600,0.166333,12.083096,0.43487,2.544190,0.156313,0.373948
3,AAPL,152.79,N/A,25.56,0.637275,36.14,0.96994,6.5000,0.853707,19.825406,0.781563,15.234777,0.91984,0.832465
4,ABBV,153.68,N/A,22.97,0.56513,17.37,0.933868,4.9100,0.777555,11.889645,0.426854,8.507717,0.685371,0.677756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,119.23,N/A,21.45,0.509018,-4.11,0.056112,5.2000,0.803607,19.742561,0.779559,9.321896,0.741483,0.577956
495,ZBH,108.68,N/A,103.73,0.981964,1.88,0.283567,2.9700,0.534068,17.439308,0.707415,5.226473,0.392786,0.57996
496,ZBRA,301.35,N/A,20.20,0.490982,5.48,0.739479,2.7900,0.50501,15.580490,0.635271,6.383423,0.476954,0.569539
497,ZION,52.22,N/A,5.23,0.074148,1.32,0.169339,2.2600,0.420842,4.217492,0.074148,2.108005,0.130261,0.173747


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [20]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

/Users/ezekielang/opt/anaconda3/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [22]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.56,N/A,-4.05,0.032064,-1.0500,0.06012,0.2764,0.016032,-21.436752,0.008016,0.975838,0.04008,0.031263
1,KSS,28.84,N/A,4.40,0.058116,0.8287,0.086172,0.1960,0.008016,3.412669,0.04008,0.975528,0.038076,0.046092
2,GPS,8.59,N/A,-45.21,0.01002,1.2900,0.156313,0.1980,0.01002,4.640291,0.096192,0.670167,0.012024,0.056914
3,AIG,52.12,N/A,4.49,0.062124,0.7258,0.074148,0.7699,0.11022,2.671851,0.028056,0.742149,0.018036,0.058517
4,GM,33.57,N/A,5.65,0.084168,0.7841,0.082164,0.3713,0.032064,1.774520,0.02004,1.470513,0.082164,0.06012
5,UNM,32.59,N/A,7.33,0.118236,0.6249,0.07014,0.5650,0.068136,3.550505,0.044088,0.537184,0.006012,0.061323
6,PVH,58.30,N/A,4.10,0.052104,0.7491,0.076152,0.4307,0.044088,4.510638,0.084168,1.033513,0.052104,0.061723
7,LNC,47.37,N/A,6.98,0.108216,0.5800,0.066132,0.4308,0.046092,5.078380,0.106212,0.430584,0.002004,0.065731
8,F,12.46,N/A,4.29,0.056112,1.0700,0.116232,0.3492,0.03006,3.763189,0.0501,1.858995,0.108216,0.072144
9,PRU,93.76,N/A,7.79,0.130261,0.7921,0.084168,0.5162,0.064128,4.885139,0.102204,0.520451,0.004008,0.076954


## Calculating the Number of Shares to Buy


In [23]:
portfolio_input()

Enter the value of your portfolio:15000


In [24]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,14.56,20,-4.05,0.032064,-1.0500,0.06012,0.2764,0.016032,-21.436752,0.008016,0.975838,0.04008,0.031263
1,KSS,28.84,10,4.40,0.058116,0.8287,0.086172,0.1960,0.008016,3.412669,0.04008,0.975528,0.038076,0.046092
2,GPS,8.59,34,-45.21,0.01002,1.2900,0.156313,0.1980,0.01002,4.640291,0.096192,0.670167,0.012024,0.056914
3,AIG,52.12,5,4.49,0.062124,0.7258,0.074148,0.7699,0.11022,2.671851,0.028056,0.742149,0.018036,0.058517
4,GM,33.57,8,5.65,0.084168,0.7841,0.082164,0.3713,0.032064,1.774520,0.02004,1.470513,0.082164,0.06012
5,UNM,32.59,9,7.33,0.118236,0.6249,0.07014,0.5650,0.068136,3.550505,0.044088,0.537184,0.006012,0.061323
6,PVH,58.30,5,4.10,0.052104,0.7491,0.076152,0.4307,0.044088,4.510638,0.084168,1.033513,0.052104,0.061723
7,LNC,47.37,6,6.98,0.108216,0.5800,0.066132,0.4308,0.046092,5.078380,0.106212,0.430584,0.002004,0.065731
8,F,12.46,24,4.29,0.056112,1.0700,0.116232,0.3492,0.03006,3.763189,0.0501,1.858995,0.108216,0.072144
9,PRU,93.76,3,7.79,0.130261,0.7921,0.084168,0.5162,0.064128,4.885139,0.102204,0.520451,0.004008,0.076954


## Formatting Excel Output

In [ ]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats for .xlsx File
Four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric



In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [ ]:
writer.save()